In [ ]:
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# 이미 clone되어 있으면 pull, 없으면 clone
import os
REPO = "https://github.com/Lithium07z/SPAG4d.git"
BRANCH = "feature/layered-inpainting"

if os.path.exists("/content/SPAG4d"):
    !git -C /content/SPAG4d pull origin {BRANCH}
else:
    !git clone -b {BRANCH} {REPO} /content/SPAG4d

%cd /content/SPAG4d

In [ ]:
# SPAG4D (포크 버전)
!pip install -q -e ".[all]"
!pip install -q torchmetrics mmengine safetensors einops py360convert

# SAM3
!pip install -q git+https://github.com/facebookresearch/sam3.git

# LaMa (simple wrapper)
!pip install -q simple-lama-inpainting

# ProPainter
!git clone -q https://github.com/sczhou/ProPainter.git /content/ProPainter
!pip install -q -r /content/ProPainter/requirements.txt

import sys
sys.path.insert(0, '/content/ProPainter')

# ── Pillow 버전 고정 (반드시 마지막에 재설치) ──────────────────────────────
# SAM3 / ProPainter requirements가 Pillow를 9.x로 다운그레이드하므로
# 모든 패키지 설치 완료 후 10.2.0으로 덮어씀
!pip install -q --force-reinstall "Pillow==10.2.0"
print("설치 완료. 아래 런타임 재시작 셀을 실행하세요.")


In [ ]:
# ⚠ 런타임 재시작 (pip 설치 직후 최초 1회만 실행)
# Pillow 10.2.0이 메모리에 반영되려면 커널 재시작이 필요합니다.
# 이 셀 실행 후 "런타임 연결됨" 상태가 되면 Step 0(cell_imports)부터 다시 실행하세요.
import os
os.kill(os.getpid(), 9)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

WEIGHTS_DIR = '/content/drive/MyDrive/360gs_weights'
os.makedirs(WEIGHTS_DIR, exist_ok=True)

# LaMa — 기존과 동일
LAMA_PATH = f'{WEIGHTS_DIR}/big-lama'
if not os.path.exists(LAMA_PATH):
    !curl -LJO https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip
    !unzip -q big-lama.zip -d {WEIGHTS_DIR}

# -----------------------------------------------
# SAM3 체크포인트 (gated access — HF 토큰 필요)
# -----------------------------------------------
# 사전 준비: https://huggingface.co/facebook/sam3 에서 Request Access 승인 후
# HF 계정 설정 → Access Tokens → New token (read 권한) 생성

from google.colab import userdata  # Colab Secrets에 저장한 토큰 사용
import os

HF_TOKEN = userdata.get('HF_TOKEN')  # Colab 좌측 🔑 Secrets 탭에 등록
os.environ['HF_TOKEN'] = HF_TOKEN

SAM3_CACHE = f'{WEIGHTS_DIR}/sam3'
os.makedirs(SAM3_CACHE, exist_ok=True)

SAM3_PATH = f'{SAM3_CACHE}/sam3.pt'
if not os.path.exists(SAM3_PATH):
    !pip install -q huggingface_hub
    from huggingface_hub import hf_hub_download
    hf_hub_download(
        repo_id="facebook/sam3",
        filename="sam3.pt",
        local_dir=SAM3_CACHE,
        token=HF_TOKEN
    )
    print("SAM3 체크포인트 다운로드 완료:", SAM3_PATH)
else:
    print("SAM3 체크포인트 캐시 사용:", SAM3_PATH)

## Step 0: 임포트 및 경로 설정

런타임 리셋 후 이 셀부터 다시 실행하면 이전 단계 결과를 Drive 캐시에서 복구합니다.

In [ ]:
import os, sys, yaml, pickle, logging
import cv2
import numpy as np
import matplotlib.pyplot as plt

# sys.path: 레포 루트 (/content/SPAG4d — cell 0에서 %cd 진입)
if '/content/SPAG4d' not in sys.path:
    sys.path.insert(0, '/content/SPAG4d')

logging.basicConfig(level=logging.INFO, format='%(levelname)s %(name)s: %(message)s')

# ── 경로 설정 (필요에 따라 수정) ─────────────────────────────────────────────
INPUT  = '/content/drive/MyDrive/360gs_input/test.mp4'   # 원본 360도 비디오
OUTPUT = '/content/drive/MyDrive/360gs_output/'          # 최종 PLY 출력 디렉토리
CACHE  = '/content/drive/MyDrive/360gs_cache/'           # 중간 결과 저장 (런타임 복구용)
os.makedirs(OUTPUT, exist_ok=True)
os.makedirs(CACHE,  exist_ok=True)

# ── configs/default.yaml ─────────────────────────────────────────────────────
with open('configs/default.yaml', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)
print('설정 로드 완료:')
for section, vals in cfg.items():
    print(f'  {section}: {vals}')

# ── pipeline 임포트 ──────────────────────────────────────────────────────────
from pipeline.segmentation import generate_masks
from pipeline.inpainting    import cubemap_inpaint_video
from pipeline.depth         import complete_depth
from pipeline.composer      import generate_layered_ply

# 시각화용 샘플 프레임 인덱스
SAMPLE_IDX = 0

print('모든 모듈 임포트 성공.')

## Step 1: 전경 마스크 생성 (SAM3)

`generate_masks(video_path, sam3_ckpt, device)` -> `dict[int, np.ndarray]`  
- 각 프레임의 전경 픽셀: 1, 배경: 0, dtype uint8  
- `CACHE/masks.pkl`에 저장 → 런타임 리셋 시 자동 복구

In [ ]:
MASKS_CACHE = os.path.join(CACHE, 'masks.pkl')

if os.path.exists(MASKS_CACHE):
    print('캐시 발견 - 마스크 로드:', MASKS_CACHE)
    with open(MASKS_CACHE, 'rb') as _f:
        masks = pickle.load(_f)
    print(f'  로드된 프레임 수: {len(masks)}')
else:
    print('마스크 생성 중 (SAM3)...')
    masks = generate_masks(INPUT, sam3_ckpt=SAM3_PATH, device='cuda')
    with open(MASKS_CACHE, 'wb') as _f:
        pickle.dump(masks, _f)
    print(f'완료: {len(masks)} 프레임 -> {MASKS_CACHE}')

### Step 1b: 마스크 시각화

In [ ]:
# 원본 프레임 읽기
_cap = cv2.VideoCapture(INPUT)
_cap.set(cv2.CAP_PROP_POS_FRAMES, SAMPLE_IDX)
_ret, _frame_bgr = _cap.read()
_cap.release()

if _ret and SAMPLE_IDX in masks:
    _frame_rgb = cv2.cvtColor(_frame_bgr, cv2.COLOR_BGR2RGB)
    _mask = masks[SAMPLE_IDX]
    _overlay = _frame_rgb.copy()
    _fg_px = _mask == 1
    _overlay[_fg_px] = (_overlay[_fg_px] * 0.4 + np.array([255, 0, 0]) * 0.6).astype(np.uint8)

    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    axes[0].imshow(_frame_rgb); axes[0].set_title(f'원본 프레임 {SAMPLE_IDX}'); axes[0].axis('off')
    axes[1].imshow(_overlay);   axes[1].set_title('마스크 오버레이 (빨간색=전경)'); axes[1].axis('off')
    plt.tight_layout(); plt.show()
    print(f'전경 픽셀: {_mask.sum():,} / {_mask.size:,} ({100*_mask.mean():.1f}%)')
else:
    print(f'프레임 또는 마스크 없음 (SAMPLE_IDX={SAMPLE_IDX})')

## Step 2: 배경 인페인팅 (LaMa + CubeMap 투영)

`cubemap_inpaint_video(video_path, masks, lama_path, device)` -> `str` (MP4 경로)  
- 경로는 `CACHE/bg_video_path.txt`에 저장

In [ ]:
BG_VIDEO_CACHE = os.path.join(CACHE, 'bg_video_path.txt')

def _bg_valid(path):
    return bool(path) and os.path.isfile(path)

bg_video = None
if os.path.exists(BG_VIDEO_CACHE):
    with open(BG_VIDEO_CACHE) as _f:
        bg_video = _f.read().strip()

if _bg_valid(bg_video):
    print('캐시 발견 - 배경 비디오:', bg_video)
else:
    print('배경 인페인팅 실행 중 (LaMa CubeMap 투영)...')
    bg_video = cubemap_inpaint_video(INPUT, masks, lama_path=LAMA_PATH, device='cuda')
    with open(BG_VIDEO_CACHE, 'w') as _f:
        _f.write(bg_video)
    print('완료:', bg_video)

### Step 2b: 인페인팅 비교 (원본 vs 배경)

In [ ]:
_cap_bg = cv2.VideoCapture(bg_video)
_cap_bg.set(cv2.CAP_PROP_POS_FRAMES, SAMPLE_IDX)
_ret_b, _bg_bgr = _cap_bg.read()
_cap_bg.release()

if _ret and _ret_b:
    _bg_rgb = cv2.cvtColor(_bg_bgr, cv2.COLOR_BGR2RGB)
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    axes[0].imshow(_frame_rgb); axes[0].set_title('원본');           axes[0].axis('off')
    axes[1].imshow(_bg_rgb);    axes[1].set_title('인페인팅 배경'); axes[1].axis('off')
    plt.tight_layout(); plt.show()
else:
    print(f'비디오 프레임 읽기 실패 (SAMPLE_IDX={SAMPLE_IDX})')

## Step 3: 배경 Depth 완성

`complete_depth(bg_video, masks, device)` -> `dict[int, np.ndarray]`  
- **입력**: `bg_video` (인페인팅 결과 MP4) -- 원본 `INPUT`이 아님  
- 마스크 영역: 주변 배경 중앙값 + delta(0.5m)로 채움  
- 결과는 `CACHE/bg_depth.pkl`에 저장

In [ ]:
DEPTH_CACHE = os.path.join(CACHE, 'bg_depth.pkl')

if os.path.exists(DEPTH_CACHE):
    print('캐시 발견 - Depth 로드:', DEPTH_CACHE)
    with open(DEPTH_CACHE, 'rb') as _f:
        bg_depth = pickle.load(_f)
    print(f'  로드된 프레임 수: {len(bg_depth)}')
else:
    # NOTE: bg_video(인페인팅 결과)를 입력으로 사용 -- INPUT(원본 비디오)이 아님
    print(f'배경 depth 완성 중 (모델: {cfg["depth"]["model"]})...')
    bg_depth = complete_depth(bg_video, masks, device='cuda')
    with open(DEPTH_CACHE, 'wb') as _f:
        pickle.dump(bg_depth, _f)
    print(f'완료: {len(bg_depth)} 프레임 -> {DEPTH_CACHE}')

### Step 3b: Depth 맵 시각화

In [ ]:
if SAMPLE_IDX in bg_depth:
    _depth_map = bg_depth[SAMPLE_IDX]
    _depth_log = np.log1p(_depth_map)
    _d_min = np.percentile(_depth_log, 1)
    _d_max = np.percentile(_depth_log, 99)
    _depth_norm = np.clip((_depth_log - _d_min) / (_d_max - _d_min + 1e-8), 0, 1)
    _depth_color_bgr = cv2.applyColorMap(
        (_depth_norm * 255).astype(np.uint8), cv2.COLORMAP_INFERNO
    )
    _depth_color = cv2.cvtColor(_depth_color_bgr, cv2.COLOR_BGR2RGB)

    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    if '_bg_rgb' in vars():
        axes[0].imshow(_bg_rgb)
    axes[0].set_title('인페인팅 배경'); axes[0].axis('off')
    axes[1].imshow(_depth_color)
    axes[1].set_title(
        f'Depth 맵 (INFERNO, {_depth_map.min():.1f}~{_depth_map.max():.1f}m)'
    )
    axes[1].axis('off')
    plt.tight_layout(); plt.show()
    print(f'Depth 통계: min={_depth_map.min():.2f}m  '
          f'median={np.median(_depth_map):.2f}m  '
          f'max={_depth_map.max():.2f}m')
else:
    print(f'Depth 없음 (SAMPLE_IDX={SAMPLE_IDX})')

## Step 4: 레이어드 3DGS PLY 생성

`generate_layered_ply(video_path, masks, bg_video_dir, bg_depth, output_dir, device)`  
-> `list[str]` (정렬된 PLY 경로 목록)  
- 프레임마다 전경 PLY + 배경 PLY를 합성해 `frame_XXXX_merged.ply`로 저장

In [ ]:
# 이미 생성된 PLY 수 확인
_existing = sorted([
    os.path.join(OUTPUT, f) for f in os.listdir(OUTPUT)
    if f.endswith('_merged.ply')
])
_n_frames = len(masks)

if len(_existing) == _n_frames:
    print(f'이미 완료 ({_n_frames}개) -- 건너뜀')
    ply_paths = _existing
else:
    if _existing:
        print(f'일부 완료 ({len(_existing)}/{_n_frames}) -- 나머지 생성 중...')
    ply_paths = generate_layered_ply(
        video_path=INPUT,
        masks=masks,
        bg_video_dir=bg_video,
        bg_depth=bg_depth,
        output_dir=OUTPUT,
        device='cuda',
    )
    print(f'완료: {len(ply_paths)} 프레임')

print(f'\n출력 파일 (총 {len(ply_paths)}개):')
for _p in ply_paths[:5]:
    _mb = os.path.getsize(_p) / 1e6
    print(f'  {os.path.basename(_p):35s}  {_mb:.1f} MB')
if len(ply_paths) > 5:
    print(f'  ... 외 {len(ply_paths)-5}개')
print('\n출력 디렉토리:', OUTPUT)